<a href="https://colab.research.google.com/github/philocifer/banner_flip_engine/blob/main/Banner_Flip_Engine_Fine_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.8 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain_community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install -qU datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.9 MB/s eta 0:00:00


In [ ]:
import os
import getpass
import nest_asyncio
import asyncio
from tqdm import tqdm
import uuid
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import wandb
from huggingface_hub import login, notebook_login

In [ ]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
class FinetuneEmbeddings:
    def __init__(self):
        self.id_set = set()
        self.model_id = "Snowflake/snowflake-arctic-embed-l"
        self.BATCH_SIZE = 10
        self.EPOCHS = 10
        self.output_path = "banner-flip-arctic-embed-l"
        self.finetuned_model_id = "philocifer/banner-flip-arctic-embed-l"
        self.push_to_hub = True

    def _get_uuid(self):
        id = str(uuid.uuid4())
        while id in self.id_set:
            id = str(uuid.uuid4())
        self.id_set.add(id)
        return id

    async def _process_document(self, question_generation_chain, document, n_questions):
        """Process a single document to generate questions and relevant context mappings.

        Args:
            document: Langchain Document object with page_content and metadata
            n_questions: Number of questions to generate per document

        Returns:
            Tuple of (questions dict, relevant_docs dict) for this document"""

        doc_questions = {}
        doc_relevant_docs = {}

        # Generate questions using LLM chain
        questions_generated = await question_generation_chain.ainvoke({
            "context": document.page_content,
            "n_questions": n_questions
        })

        # Process each generated question line
        for question in questions_generated.content.split("\n"):
            # Create unique ID for question
            question_id = self._get_uuid()

            # Remove numbering from question string and clean whitespace
            doc_questions[question_id] = "".join(question.split(".")[1:]).strip()

            # Link question to document's UUID
            doc_relevant_docs[question_id] = [document.metadata["id"]]

        return doc_questions, doc_relevant_docs

    async def _create_questions(self, question_generation_chain, documents, n_questions):
        """Orchestrate parallel processing of documents to generate questions.

        Args:
            documents: List of Langchain Document objects
            n_questions: Number of questions per document

        Returns:
            Tuple of aggregated (questions dict, relevant_docs dict)"""

        questions = {}
        relevant_docs = {}

        # Create async tasks for all documents
        tasks = [self._process_document(question_generation_chain, doc, n_questions) for doc in documents]

        # Process tasks with progress bar
        for task in tqdm(asyncio.as_completed(tasks),
                        total=len(documents),
                        desc="Processing Documents"):
            doc_questions, doc_relevant_docs = await task

            # Aggregate results from all documents
            questions.update(doc_questions)
            relevant_docs.update(doc_relevant_docs)

        return questions, relevant_docs

    async def _format_dataset(self, documents, questions, relevant_docs):
        corpus = {item.metadata["id"] : item.page_content for item in documents}
        dataset = {
            "questions": questions,
            "relevant_docs": relevant_docs,
            "corpus": corpus
        }
        return dataset

    async def run(self):
        nest_asyncio.apply()

        loader = TextLoader('/content/drive/MyDrive/data/enhanced_store_data_200.txt')
        docs = loader.load()
        text_splitter = CharacterTextSplitter(
            separator="="*80,
            chunk_size=1000,
            chunk_overlap=0,
            is_separator_regex=False
        )
        training_documents = text_splitter.split_documents(docs)

        for document in training_documents:
            id = self._get_uuid()
            document.metadata["id"] = id

        train_split_documents = training_documents[:len(training_documents) - 50]
        val_split_documents = training_documents[len(training_documents) - 50:]

        qa_chat_model = ChatOpenAI(model="gpt-4o-mini",temperature=0)

        qa_prompt = """\
        Given the following context, you must generate questions based on only the provided context.

        You are to generate {n_questions} questions which should be provided in the following format:

        1. QUESTION #1
        2. QUESTION #2
        ...

        Context:
        {context}
        """
        qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

        question_generation_chain = qa_prompt_template | qa_chat_model

        train_questions, train_relevant_contexts = await self._create_questions(question_generation_chain, train_split_documents, 2)
        val_questions, val_relevant_contexts = await self._create_questions(question_generation_chain, val_split_documents, 2)

        train_dataset = await self._format_dataset(train_split_documents, train_questions, train_relevant_contexts)
        val_dataset = await self._format_dataset(val_split_documents, val_questions, val_relevant_contexts)

        embedding_model = SentenceTransformer(self.model_id)

        train_corpus = train_dataset['corpus']
        train_queries = train_dataset['questions']
        train_relevant_docs = train_dataset['relevant_docs']

        examples = []
        for query_id, query in train_queries.items():
            doc_id = train_relevant_docs[query_id][0]
            text = train_corpus[doc_id]
            example = InputExample(texts=[query, text])
            examples.append(example)

        loader = DataLoader(examples, shuffle=True, batch_size=self.BATCH_SIZE)

        matryoshka_dimensions = [768, 512, 256, 128, 64]
        inner_train_loss = MultipleNegativesRankingLoss(embedding_model)
        train_loss = MatryoshkaLoss(embedding_model, inner_train_loss, matryoshka_dims=matryoshka_dimensions)

        val_corpus = val_dataset['corpus']
        val_queries = val_dataset['questions']
        val_relevant_docs = val_dataset['relevant_docs']

        evaluator = InformationRetrievalEvaluator(val_queries, val_corpus, val_relevant_docs)

        wandb.init(mode="disabled")
        warmup_steps = int(len(loader) * self.EPOCHS * 0.1)

        print("Training model...")
        embedding_model.fit(
            train_objectives=[(loader, train_loss)],
            epochs=self.EPOCHS,
            warmup_steps=warmup_steps,
            output_path=self.output_path,
            show_progress_bar=True,
            evaluator=evaluator,
            evaluation_steps=50
        )

        print("Model trained successfully.")

        if self.push_to_hub:
            login()
            embedding_model.push_to_hub(self.finetuned_model_id, exist_ok=True)
            print("Model pushed to hub successfully.")

        return embedding_model

In [ ]:
finetune_embeddings = FinetuneEmbeddings()
await finetune_embeddings.run()

Processing Documents: 100%|██████████| 50/50 [00:03<00:00, 16.34it/s]


Training model...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
30,No log,No log,0.840000,0.920000,0.960000,0.980000,0.840000,0.306667,0.192000,0.098000,0.840000,0.920000,0.960000,0.980000,0.911102,0.888762,0.889222
50,No log,No log,0.840000,0.920000,0.950000,0.980000,0.840000,0.306667,0.190000,0.098000,0.840000,0.920000,0.950000,0.980000,0.910567,0.888190,0.888680
60,No log,No log,0.830000,0.920000,0.950000,0.980000,0.830000,0.306667,0.190000,0.098000,0.830000,0.920000,0.950000,0.980000,0.905795,0.881762,0.882391
90,No log,No log,0.830000,0.940000,0.970000,0.990000,0.830000,0.313333,0.194000,0.099000,0.830000,0.940000,0.970000,0.990000,0.914852,0.890095,0.890480
100,No log,No log,0.840000,0.940000,0.970000,0.990000,0.840000,0.313333,0.194000,0.099000,0.840000,0.940000,0.970000,0.990000,0.919852,0.896762,0.897262
120,No log,No log,0.840000,0.940000,0.970000,0.990000,0.840000,0.313333,0.194000,0.099000,0.840000,0.940000,0.970000,0.990000,0.918543,0.895095,0.895466
150,No log,No log,0.850000,0.930000,0.960000,0.990000,0.850000,0.310000,0.192000,0.099000,0.850000,0.930000,0.960000,0.990000,0.920795,0.898429,0.898762
180,No log,No log,0.850000,0.930000,0.960000,0.980000,0.850000,0.310000,0.192000,0.098000,0.850000,0.930000,0.960000,0.980000,0.917233,0.896762,0.897957
200,No log,No log,0.850000,0.930000,0.960000,0.980000,0.850000,0.310000,0.192000,0.098000,0.850000,0.930000,0.960000,0.980000,0.917233,0.896762,0.897839
210,No log,No log,0.850000,0.930000,0.960000,0.980000,0.850000,0.310000,0.192000,0.098000,0.850000,0.930000,0.960000,0.980000,0.917233,0.896762,0.897753


Model trained successfully.


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Model pushed to hub successfully.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
